# Process

I will try (and fail probably) to put everything into one.

# Import Libraries

In [113]:
import time
from qwikidata.sparql  import return_sparql_query_results
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import urllib.request
import json
import string
from difflib import SequenceMatcher
from viapy.api import ViafAPI

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [114]:
location_to_save = "data/final_files/"

# 1. Download all WikiData entries
Goal: To download (using SparQL queries) all WikiData entries that have VIAF ids.

A. Get all WikiData entries with VIAF ID.

In [3]:
query_string = """
        SELECT DISTINCT ?item ?viaf # return QID and VIAF ID
WHERE 
{ ?item wdt:P214 ?viaf. # select all the items in WikiData that have VIAF ID

}
        """
start = time.time()
items_with_viaf = return_sparql_query_results(query_string) # run the query and get results
print("It took "+str(np.round(time.time()-start,2))+ " seconds.")

It took 64.95 seconds.


B. Process results.

In [4]:
items_with_viaf = pd.DataFrame(items_with_viaf["results"]["bindings"])
items_with_viaf["item"] = items_with_viaf["item"].apply(lambda x: x["value"].split("/")[-1]) # keep only QID
items_with_viaf["viaf"] = items_with_viaf["viaf"].apply(lambda x: x["value"]) # keep only VIAF ID
items_with_viaf.columns = ["QID", "viaf_id"] # rename columns

In [5]:
items_with_viaf.head()

,QID,viaf_id
0,Q270705,16565
1,Q289693,135150789
2,Q481146,14775085
3,Q507746,6650
4,Q568833,158264550


In [6]:
print("In WikiData, there are "+str(len(items_with_viaf))+" items with VIAF ID.")

In WikiData, there are 2822447 items with VIAF ID.


C. Save results.

In [ ]:
# # save with Pickle
# with open(location_to_save + "items_with_viaf_wikidata.pkl", "wb") as output_file:
#     pkl.dump(items_with_viaf, output_file)

# 2. Recreate Fairbook dataset
Goal: To cut down the dataset the same way that they did in FairBook.

A. Read files.

In [115]:
dataset = pd.read_csv("data/ratings_books.csv", low_memory=False).fillna("")
dataset.columns = ['User-ID', 'ISBN', 'Book-Rating']

B. Print statistics.

In [116]:
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['ISBN'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 105283
> No. of Books: 340556
> No. of Interaction: 1149780


C. Filter dataset.

In [9]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

C.1. Remove implicit (i.e. 0) ratings.

In [10]:
dataset = filter_rows_by_values(dataset, "Book-Rating", [0])

# statistics on explicit dataset
print("Explicit dataset statistics: ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['ISBN'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Explicit dataset statistics: 
> No. of users: 77805
> No. of Books: 185973
> No. of Interaction: 433671


C.2. Remove:
1. Users with more than 200 ratings
2. Users with less than 5 rarings
3. Items with less than 5 ratings

In [11]:
# To check if there is any user with more than 200 interaction in the preprocessed dataset
# The correct output will be zero
uid_value_counts = dataset['User-ID'].value_counts()
print("The number of users with more than 200 interactions:", len(uid_value_counts[uid_value_counts > 200]))

The number of users with more than 200 interactions: 144


In [12]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['User-ID'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['Count'] = dataset['User-ID'].map(users_counts)

In [13]:
dataset = filter_rows_by_values(dataset, "Count", list(range(200, max(dataset['Count']) + 1)))

In [14]:
# statistics on explicit dataset after removing users with more than 200 int.
print(f"New dataset statistics (users with interactions < {200}): ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['ISBN'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

New dataset statistics (users with interactions < 200): 
> No. of users: 77660
> No. of Books: 156891
> No. of Interaction: 364245


In [15]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < 5 interactions: ")
    # user side fewer than ds_rate cheking
    uid_value_counts = dataset['User-ID'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < 5].count()
    print(f"No. of users < 5 ineractions: {user_interaction}")

    users_counts = dataset['User-ID'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['User-ID'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(5)))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['ISBN'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < 5].count()
    print(f"No. of items < 5 ineractions: {item_interaction}")

    items_counts = dataset['ISBN'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['ISBN'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(5)))

The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 63585
No. of items < 5 ineractions: 119254
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 5825
No. of items < 5 ineractions: 1594
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 432
No. of items < 5 ineractions: 208
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 69
No. of items < 5 ineractions: 45
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 19
No. of items < 5 ineractions: 17
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 11
No. of items < 5 ineractions: 13
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 6
No. of items < 5 ineractions: 2
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 2
No. of items < 

D. Print statistics after preprocessing.

In [16]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['User-ID'].unique())}")
print(f"No. of Books: {len(dataset['ISBN'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 6358
No. of Books: 6921
No. of Interaction: 88552


E. Save file.

In [17]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['Count']
dataset.to_csv(location_to_save+"fairbook_ratings.csv")

# 3. First look in the data

A. Read book file.

In [18]:
filename = "data/items_books.csv" # Books-Crossing books
books = pd.read_csv(filename, low_memory = False).drop(["Image-URL-S", "Image-URL-M","Image-URL-L"], axis=1) # read books and remove images
books.columns = ["ISBN", "title", "author", "year", "publisher"] # rename columns to simplify

In [19]:
print("There are "+str(len(books))+" books.")

There are 271360 books.


In [20]:
books.head()

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


B. Choose "unique" authors based on exact author name.

In [21]:
authors = pd.DataFrame(books.author.unique(), columns=["author"]).dropna().reset_index().drop("index", axis=1) # find unique authors and remove nan values

In [22]:
print("There are "+str(len(authors))+" unique authors.")

There are 102023 unique authors.


In [23]:
authors.head()

,author
0,Mark P. O. Morford
1,Richard Bruce Wright
2,Carlo D'Este
3,Gina Bari Kolata
4,E. J. W. Barber


C. Statistics.

C.1 Non-latin named authors

In [24]:
def isEnglish(s): # function to check if a name is latin
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [25]:
authors["author_latin"] = authors.author.apply(lambda x : isEnglish(x))
num_latin_authors = len(authors[authors.author_latin == True])

In [26]:
print("There are ", num_latin_authors,"latin authors out of", str(len(authors))+".")

There are  100642 latin authors out of 102023.


In [27]:
del authors["author_latin"]

C.2 Author entries with a single name.

In [28]:
authors["author_single"] = authors.author.apply(lambda x: len(x.split(" "))==1)
num_single_named_authors = len(authors[authors.author_single == True])

In [29]:
print("There are ", num_single_named_authors,"single named authors out of", str(len(authors))+".")

There are  2502 single named authors out of 102023.


# 4. Access Google Books API
Goal: To use the ISBNs from the dataset to get the author and title of the book.

A. Function to access the API.

In [30]:
def get_author_title(ISBN):
    base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

        
    with urllib.request.urlopen(base_api_link + ISBN) as f:
        text = f.read()
    decoded_text = text.decode("utf-8")
    obj = json.loads(decoded_text) # deserializes decoded_text to a Python object
    try:
        volume_info = obj["items"][0] 
    except: 
        
        #print(obj)
        return("","")
    title = volume_info["volumeInfo"]["title"]
    try:
        authors = volume_info["volumeInfo"]["authors"]
        
    except:
        #print(volume_info["volumeInfo"])
        return("",title)

    return(authors,title)

B. Initialize the entries.

In [31]:
books["alt_title"] = ""
books["alt_author"] = ""

C. Run the requests.

In [ ]:
start = time.time()
for index,row in books.iloc[271000:].iterrows():
    if row["alt_author"]=="":
        ISBN = row["ISBN"]
        print(row.title)
        statement = True
        i=0
        while statement:

            try:
                authors, title = get_author_title(ISBN)
                statement = False
            except:
                if i>50:
                    statement = False
                    print("IT FAILED MORE THAN 50 TIMES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                time.sleep(0.1)
            i+=1  

        if type(authors)==list:
            authors = "|".join(authors)
        books.at[index, "alt_title"] = title
        books.at[index, "alt_author"] = authors
    if index%100==0:
        print(time.time()-start, index)
        start = time.time()
        time.sleep(5)
        if index%1000==0:
            print("SAVE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            books.to_csv(location_to_save+"items_books_some_isbn.csv")

In [ ]:
books.to_csv(location_to_save+"items_books_some_isbn.csv")

# 5. Analyze results from Google Books API
Goal: To check the structure and completeness of the Google Books information.

<b>A. Read file.</b>

In [32]:
books = pd.read_csv(location_to_save + "items_books_some_isbn.csv", low_memory = False, index_col=0).fillna("")

<b>B. Check completeness.</b>

In [35]:
no_results_authors = len(books[(books.alt_author=="")].drop_duplicates(subset = "author"))
no_results_books = len(books[(books.alt_author=="")])

In [36]:
print("For", no_results_authors,"unique authors, we couldn't match the ISBN of at least one of their books, out of",len(books.drop_duplicates(subset = "author")), "authors.")
print("For", no_results_books,"books, we couldn't match the ISBN, out of", len(books), "books.")

For 6511 unique authors, we couldn't match the ISBN of at least one of their books, out of 102024 authors.
For 10134 books, we couldn't match the ISBN, out of 271360 books.


<b>C. Compare Book-Crossing with Google Books.</b>

<b>C.1 First estimation: Naive. </b>

In [37]:
first_estimation = len(books[books.author!=books.alt_author])
print("At first glance, we didn't find the same author for", first_estimation,"books out of ",str(len(books))+". However, it needs more processing.")

At first glance, we didn't find the same author for 92052 books out of  271360. However, it needs more processing.


<b>C.2 Second estimation: Simplify author names.</b>

We need a function that:
1. Removes spaces
2. Removes punctuation
3. Makes lowercase

<b>This way, we can more properly compare the strings.</b>

In [38]:
def simplify(name):
    name = name.replace(" ","").translate(str.maketrans('', '', string.punctuation)).lower()
    return name

In [39]:
simplify("Savvina G. Daniil")

'savvinagdaniil'

We create a column called "correct_author" where we replace it with the alt_author if simplified it is the same as the book crossing author. 

In [40]:
books["correct_author"] = ''
for index,row in books.iterrows():
    if row.alt_author!="":
        if simplify(row.author)==simplify(row.alt_author):
            books.at[index, "correct_author"] = row.alt_author

In [41]:
second_estimation = len(books[books.correct_author==""])
print("With the simplifying, we didn't find the same author for", second_estimation,"books out of ",str(len(books))+". However, it needs even more processing.")

With the simplifying, we didn't find the same author for 81731 books out of  271360. However, it needs even more processing.


#### C.3 Third estimation: Account for multiple authors.

Since in alt_author we collect all authors, we make sure that we consider if any of them (separated by "|") is the same as author.

In [42]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        if row.alt_author!="": # if we got ISBN result
            alt_author = row.alt_author
            bag_of_alt_authors = alt_author.split("|")
            simplified_bag_of_alt_authors = [simplify(x) for x in bag_of_alt_authors]
            author = simplify(row.author)
            for i in range(len(bag_of_alt_authors)):
                if author==simplified_bag_of_alt_authors[i]:
                    books.at[index, "correct_author"] = bag_of_alt_authors[i]

In [43]:
third_estimation = len(books[books.correct_author==""])
print("With accounting for bag of authors, we didn't find the same author for", third_estimation,"books out of ",str(len(books))+". However, it needs even more processing.")

With accounting for bag of authors, we didn't find the same author for 50833 books out of  271360. However, it needs even more processing.


#### C.4 Fourth estimation: Reverse names.

I noticed that some names are reversed, e.g. Charles Glass and Glass Charles are not considered the same name by my processing. So I add an extra layer: sort the characters of the name. 

In [44]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [sorted(simplify(x)) for x in bag_of_alt_authors]
        author = sorted(simplify(row.author))
        for i in range(len(bag_of_alt_authors)):
            if author==simplified_bag_of_alt_authors[i]:
                books.at[index, "correct_author"] = bag_of_alt_authors[i]

In [45]:
fourth_estimation = len(books[books.correct_author==""])
print("With the sorting trick, we didn't find the same author for", fourth_estimation,"books out of ",str(len(books))+". However, it needs even more processing.")

With the sorting trick, we didn't find the same author for 49731 books out of  271360. However, it needs even more processing.


#### C.5 Fifth estimation: String similarity. 

At this point, we can no longer rely on simply the literal comparison between strings. We need to introduce a similarity measure between them.

In [46]:
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [47]:
similarity("Savvina Daniil", "Savvina G. Daniil"), similarity("Savvina Daniil", "John Williams")

(0.9032258064516129, 0.2962962962962963)

I will consider every similarity over 0.7 to be the same name.

In [48]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [sorted(simplify(x)) for x in bag_of_alt_authors]
        author = sorted(simplify(row.author))
        
        max_sim = 0
        max_i = -1000
        
        for i in range(len(bag_of_alt_authors)):
            sim = similarity(author, simplified_bag_of_alt_authors[i])
            if sim>max_sim:
                max_sim = sim
                max_i = i
        
        if max_sim>=0.7:
            books.at[index, "correct_author"] = bag_of_alt_authors[max_i]

In [49]:
fifth_estimation = len(books[books.correct_author==""])
print("With the similarity threshold, we didn't find the same author for", fifth_estimation,"books out of ",str(len(books))+". At this point, it is hard to further progress.")

With the similarity threshold, we didn't find the same author for 22019 books out of  271360. At this point, it is hard to further progress.


In [105]:
print("FINAL SUM UP")
print("Books:                           |", len(books))
print("Books with ISBN results:         |", len(books[books.alt_author!=""]))
print("Books with matching ISBN results:|", len(books[books.correct_author!=""]))

FINAL SUM UP
Books:                           | 271360
Books with ISBN results:         | 261226
Books with matching ISBN results:| 249341


#### C.6 Save first author from GB

We may need to use them when running the viaf API.

In [50]:
books["alt_first_author"] = books.alt_author.apply(lambda x: x.split("|")[0])

In [51]:
books.head()

,ISBN,title,author,year,publisher,alt_title,alt_author,correct_author,alt_first_author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,Decision in Normandy,Carlo D'Este,Carlo D'Este,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,Flu,Gina Bari Kolata,Gina Bari Kolata,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,The Mummies of Ürümchi,E. J. W. Barber,E. J. W. Barber,E. J. W. Barber


In [52]:
books.to_csv(location_to_save+"items_books_some_isbn.csv")

# 6. Access VIAF API
For the "matching" authors.

In [53]:
books = pd.read_csv(location_to_save+"items_books_some_isbn.csv", low_memory = False, index_col=0).fillna("")

A. Function to access the API.

In [54]:
def get_viaf_entry(author_name):
    #print(author_name)
    viaf = ViafAPI()
    viaf_id = ""
    viaf_name = ""
    potential_entries = viaf.suggest(author_name)[:5]
    if len(potential_entries)==0:
        print("No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    for entry in potential_entries:
        if entry["nametype"]=="personal":
            #print(entry["term"])
            try:
                viaf_id = entry["viafid"]
                viaf_name = entry["term"]
                #print("We found viaf_id!")
            except:
                print(author_name + " has no Viaf ID.")
            break
        print("ISSUE! We didn't find for", author_name)
    return viaf_id, viaf_name

B. Initialize the entries.

In [55]:
books["viaf_id"] = ""
books["viaf_name"] = ""

C. Define unique authors.

The purpose is to not rerun the API again for the same "correct author" name.

In [69]:
definitive_unique_authors = books[books.correct_author!=""].drop_duplicates(subset="correct_author")

D. Run the requests.

In [71]:
start = time.time()
i=0
for index, row in definitive_unique_authors.iloc[86000:].iterrows():
    if row.viaf_id=="":
        author = row.correct_author
        viaf_id, viaf_name = get_viaf_entry(author)
        if viaf_id == "":
            print("try normal author name")
            viaf_id, viaf_name = get_viaf_entry(row.author)
        books.at[index, "viaf_id"] = viaf_id
        books.at[index, "viaf_name"] = viaf_name
        definitive_unique_authors.at[index, "viaf_id"] = viaf_id
        definitive_unique_authors.at[index, "viaf_name"] = viaf_name
    if i%500==0:
        print(index, time.time()-start)
        books.to_csv(location_to_save+"items_books_after_viaf.csv")
    i+=1

270619 0.0006613731384277344
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
ISSUE! We didn't find for Personal Stamp Exchange
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try normal author name
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!

In [75]:
books.to_csv(location_to_save+"items_books_after_viaf.csv")

# 7. Complete the VIAF enriched dataset

Given that I dropped duplicates for "correct_author", I now need to fill the entire books dataset with the viaf information I got.

A. Read file.

In [163]:
books = pd.read_csv(location_to_save+"items_books_after_viaf.csv", dtype = object, index_col=0).fillna("")

B. Keep books with matching ISBN results and then authors with viaf results.

In [164]:
books_with_matching_ISBN_results = books[books.correct_author!=""]

In [165]:
authors_with_viaf = books_with_matching_ISBN_results[["correct_author", "viaf_id", "viaf_name"]].loc[books_with_matching_ISBN_results.viaf_id!=""]

In [166]:
books_with_matching_ISBN_results = books_with_matching_ISBN_results.drop(["viaf_id", "viaf_name"], axis =1)

C. Keep books with mathcing ISBN results and viaf results.

In [167]:
books_with_matching_ISBN_results_and_viaf = pd.merge(books_with_matching_ISBN_results.reset_index(), authors_with_viaf, on="correct_author").set_index("index")

D. Fill the books dataset with these results.

In [168]:
books.loc[books_with_matching_ISBN_results_and_viaf.index] = books_with_matching_ISBN_results_and_viaf

E. Sum up.

In [183]:
print("FINAL SUM UP                              | Found  | Remaining")
print("------------------------------------------|--------|----------")
print("Books:                                    |", len(books),"|", len(books) - len(books))
print("Books with ISBN results:                  |", len(books[books.alt_author!=""]),"|", len(books[books.alt_author==""]))
print("Books with matching ISBN results:         |", len(books[books.correct_author!=""]),"|", len(books[books.correct_author==""]))
print("Books with matching ISBN and VIAF results:|", len(books[books.viaf_id!=""]),"|", len(books[books.viaf_id==""]))

FINAL SUM UP                              | Found  | Remaining
------------------------------------------|--------|----------
Books:                                    | 271360 | 0
Books with ISBN results:                  | 261226 | 10134
Books with matching ISBN results:         | 249341 | 22019
Books with matching ISBN and VIAF results:| 244457 | 26903


In [170]:
books.to_csv(location_to_save+"items_books_after_viaf.csv")

# 8. Rerun VIAF for books without matching ISBN results

There are ~5K books with matcing authors for whom we didn't find VIAF id. However, we didn't try books without matching authors, or even any Google Books results at all.

In [196]:
books = pd.read_csv(location_to_save+"items_books_after_viaf.csv", dtype = object, index_col=0).fillna("")

A. Define authors to run: unique in terms of combination of author and GB first author.

In [197]:
unique_unfilled_authors = books[books.correct_author==""].drop_duplicates(["author", "alt_first_author"])

In [198]:
len(unique_unfilled_authors)

15732

B. Run VIAF again.

In [ ]:
start = time.time()
i=0
for index, row in unique_unfilled_authors.iterrows():
    if books.at[index,"viaf_id"]=="":
        author = row.author
        alt_first_author = row.alt_first_author
        viaf_id_author, viaf_name_author = get_viaf_entry(author)
        if alt_first_author!="":
            viaf_id_alt_first_author, viaf_name_alt_first_author = get_viaf_entry(alt_first_author)
        else:
            viaf_id_alt_first_author, viaf_name_alt_first_author = "", ""
        books.at[index, "viaf_id"] = viaf_id_author+"||"+viaf_id_alt_first_author
        books.at[index, "viaf_name"] = viaf_name_author+"||"+viaf_name_alt_first_author
        unique_unfilled_authors.at[index, "viaf_id"] = viaf_id_author+"||"+viaf_id_alt_first_author
        unique_unfilled_authors.at[index, "viaf_name"] = viaf_name_author+"||"+viaf_name_alt_first_author
        if i%500==0:
            print(index, time.time()-start)
            books.to_csv(location_to_save+"items_books_after_viaf.csv")
        i+=1

13 0.41941165924072266
ISSUE! We didn't find for The Onion
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for LTD
ISSUE! We didn't find for LTD
ISSUE! We didn't find for LTD
ISSUE! We didn't find for LTD
ISSUE! We didn't find for LTD
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Guinness World Records
ISSUE! We didn't find for Guinness World Records
ISSUE! We didn't find for Guinness World Records
ISSUE! We didn't find for Guinness World Records
ISSUE! We didn't find for Guinness World Records
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Collectif
ISSUE! We didn't find for Collectif
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Readers Digest
ISSUE! We didn't find for Readers Digest
ISSUE! We didn

ISSUE! We didn't find for Ladybird Books
ISSUE! We didn't find for Ladybird Books
ISSUE! We didn't find for Ladybird Books
ISSUE! We didn't find for Ladybird Books
ISSUE! We didn't find for Ladybird Books
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Rough Guides
ISSUE! We didn't find for Rough Guides
ISSUE! We didn't find for Rough Guides
ISSUE! We didn't find for Roug

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Sting (Musician)
ISSUE! We didn't find for Sting (Musician)
ISSUE! We didn't find for Sting (Musician)
ISSUE! We didn't find for Sting (Musician)
ISSUE! We didn't find for Herman Miller
ISSUE! We didn't find for Herman Miller
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
N

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Concordia Publishing House
ISSUE! We didn't find for Concordia Publishing House
ISSUE! We didn't find for Concordia Publishing House
ISSUE! We didn't find for Concordia Publishing House
ISSUE! We didn't find for Concordia Publishing House
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!

ISSUE! We didn't find for Scholastic Books
ISSUE! We didn't find for Scholastic Books
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Press
ISSUE! We didn't find for Golden Press
ISSUE! We didn't find for Golden Press
ISSUE! We didn't find for Golden Press
ISSUE! We didn't find for Golden Press
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We didn't find for Golden Books
ISSUE! We d

...

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Brockhampton
ISSUE! We didn't find for Brockhampton
ISSUE! We didn't find for Brockhampton
ISSUE! We didn't find for Brockhampton
ISSUE! We didn't find for Brockhampton
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Various
ISSUE! We didn't find for Various
ISSUE! We didn't find for Various
ISSUE! We didn't find for Various
ISSUE! We didn't find for Various
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Random House
ISSUE! We didn't find for Random House
ISSUE! We didn't find for Random House
ISSUE! We didn't find for Random House
ISSUE! We didn't find for Random House
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No resul

ISSUE! We didn't find for National Geographic Society (U.S.)
ISSUE! We didn't find for National Geographic Society (U.S.)
ISSUE! We didn't find for National Geographic Society (U.S.)
ISSUE! We didn't find for National Geographic Society (U.S.)
ISSUE! We didn't find for National Geographic Society (U.S.)
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Nelson Bibles
ISSUE! We didn't find for Hal Leonard Publishing Corporation
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Broadman & Holman Publishers
ISSUE! We didn't find for Broadman & Holman Publishers
ISSUE! We didn't find for Broadman & Holman Publishers
ISSUE! We didn't find for Broadman & Holman Publishers
ISSUE! We didn't find for Broadman & Holman Publishers
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Time Magazine
ISSUE! We

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Prevention Magazine Health Books
ISSUE! We didn't find for Prevention Magazine Health Books
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for 편집부
ISSUE! We didn't find for 편집부
ISSUE! We didn't find for 편집부
ISSUE! We didn't find for 편집부
ISSUE! We didn't find for 편집부
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Parragon Book Service Limited
ISSUE! We didn't find for Microsoft Corporation
ISSUE! We didn't find for Microsoft Corporation
ISSUE! We didn't find for Microsoft Corporation
ISSUE! We didn't find for Microsoft Corporation
ISSUE! We didn't find for Microsoft Corporation
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No r

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Dorling Kindersley Publishing
ISSUE! We didn't find for Dorling Kindersley Publishing
ISSUE! We didn't find for Dorling Kindersley Publishing
ISSUE! We didn't find for Dorling Kindersley Publishing
ISSUE! We didn't find for Dorling Kindersley Publishing
ISSUE! We didn't find for DK Publishing, Inc
ISSUE! We didn't find for DK Publishing, Inc
ISSUE! We didn't find for DK Publishing, Inc
ISSUE! We didn't find for DK Publishing, Inc
ISSUE! We didn't find for DK Publishing, Inc
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Arizona Center for Medieval and Renaissa
ISSUE! We didn't find for Arizona Center for Medie

ISSUE! We didn't find for Fisher-Price
ISSUE! We didn't find for Fisher-Price
ISSUE! We didn't find for Fisher-Price
ISSUE! We didn't find for Fisher-Price
ISSUE! We didn't find for Fisher-Price
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Random House Publishing Group
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ISSUE! We didn't find for Bread Loaf Writers' Conference of Middlebury College
ISSUE! We didn't find for Bread Loaf Writers' Conference of Middlebury College
ISSUE! We didn't find for Bread Loaf Writers' Conference of Middlebury College
ISSUE! We didn't find for Dempsey Parr
ISSUE! We didn't find for La Mama Theatre
ISSUE! We didn't find for Klutz Press
ISSUE! We didn't find for Klutz Press
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
No results!!!!!!!!!!!!!!!!!!!!!